# Version 1

In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import gymnasium as gym
from gymnasium import spaces
import cv2
import os
import tqdm
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚡ HARDWARE: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# --- CONFIGURATION ---
HYPER_PARAMS = {
    "GRID_W": 10,
    "MAX_STEPS": 100,
    "EPISODES": 100, # More episodes for genius emergence
    "BATCH_SIZE": 64,
    "VECTOR_DIM": 32, # State vector size
}

# --- CELL 1: THE TITAN CRAFT ENVIRONMENT (TURBO) ---
class TitanCraftEnv(gym.Env):
    def __init__(self):
        super(TitanCraftEnv, self).__init__()
        self.action_space = spaces.Discrete(6)
        # Vector: [AgentPos(2), Inv(4), Tools(2), NearTree(2), NearRock(2), NearIron(2), NearDiamond(2), StepCount(1)]
        self.observation_space = spaces.Box(low=0, high=10, shape=(HYPER_PARAMS["VECTOR_DIM"],), dtype=np.float32)
        self.grid_size = HYPER_PARAMS["GRID_W"]

    def reset(self):
        self.agent_pos = np.array([5, 5])
        self.inventory = {"wood": 0, "stone": 0, "iron": 0, "diamond": 0}
        self.tools = {"table": False, "pick_level": 0}
        self.steps = 0
        self.map_objects = []
        for _ in range(3): self.map_objects.append({"type": "tree", "pos": self._rand_pos()})
        for _ in range(3): self.map_objects.append({"type": "rock", "pos": self._rand_pos()})
        for _ in range(2): self.map_objects.append({"type": "iron", "pos": self._rand_pos()})
        self.map_objects.append({"type": "diamond", "pos": self._rand_pos()})
        return self._get_obs()

    def _rand_pos(self):
        return np.random.randint(0, self.grid_size, size=2)

    def _get_obs(self):
        # Flattened state for fast learning
        obs = [self.agent_pos[0]/10.0, self.agent_pos[1]/10.0]
        obs += [self.inventory[k]/5.0 for k in ["wood", "stone", "iron", "diamond"]]
        obs += [1.0 if self.tools["table"] else 0.0, self.tools["pick_level"]/3.0]

        # Add relative pos of nearest objects of each type
        for t in ["tree", "rock", "iron", "diamond"]:
            nearest_dist = 100
            rel_pos = np.array([0, 0])
            for obj in self.map_objects:
                if obj["type"] == t and obj["pos"][0] != -1:
                    d = np.sum(np.abs(self.agent_pos - obj["pos"]))
                    if d < nearest_dist:
                        nearest_dist = d
                        rel_pos = obj["pos"] - self.agent_pos
            obs += [rel_pos[0]/10.0, rel_pos[1]/10.0]

        # Pad to VECTOR_DIM
        obs += [self.steps / 250.0]
        while len(obs) < HYPER_PARAMS["VECTOR_DIM"]: obs.append(0.0)
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        self.steps += 1
        reward = -0.05 # Existential pressure
        done = False
        info = {}

        move = {0: [0,-1], 1: [0,1], 2: [-1,0], 3: [1,0]}
        if action < 4:
            old_dist = self._min_dist_to_any()
            new_pos = self.agent_pos + np.array(move[action])
            self.agent_pos = np.clip(new_pos, 0, self.grid_size-1)
            new_dist = self._min_dist_to_any()
            # Reward Shaping: Better to move toward objects
            if new_dist < old_dist: reward += 0.05

        elif action == 4: # Interact
            for obj in self.map_objects:
                if np.array_equal(self.agent_pos, obj["pos"]):
                    if obj["type"] == "tree":
                        self.inventory["wood"] += 1
                        reward += 2.0 # High value for foundation
                        obj["pos"] = np.array([-1, -1])
                        info["log"] = "🪓 Chopped Wood"
                    elif obj["type"] == "rock" and self.tools["pick_level"] >= 1:
                        self.inventory["stone"] += 1
                        reward += 3.0
                        obj["pos"] = np.array([-1, -1])
                        info["log"] = "⛏️ Mined Stone"
                    elif obj["type"] == "iron" and self.tools["pick_level"] >= 2:
                        self.inventory["iron"] += 1
                        reward += 5.0
                        obj["pos"] = np.array([-1, -1])
                        info["log"] = "⛓️ Mined Iron"
                    elif obj["type"] == "diamond" and self.tools["pick_level"] >= 3:
                        self.inventory["diamond"] += 1
                        reward += 100.0
                        done = True
                        info["log"] = "💎 FOUND DIAMOND!"

        elif action == 5: # Craft
            if self.inventory["wood"] >= 2 and not self.tools["table"]:
                self.inventory["wood"] -= 2
                self.tools["table"] = True
                reward += 5.0
                info["log"] = "🔨 Crafted Table"
            elif self.tools["table"] and self.inventory["wood"] >= 2 and self.tools["pick_level"] == 0:
                self.inventory["wood"] -= 2
                self.tools["pick_level"] = 1
                reward += 5.0
                info["log"] = "🗡️ Crafted Wood Pick"
            elif self.tools["table"] and self.inventory["stone"] >= 3 and self.tools["pick_level"] == 1:
                self.inventory["stone"] -= 3
                self.tools["pick_level"] = 2
                reward += 10.0
                info["log"] = "⚒️ Crafted Stone Pick"
            elif self.tools["table"] and self.inventory["iron"] >= 3 and self.tools["pick_level"] == 2:
                self.inventory["iron"] -= 3
                self.tools["pick_level"] = 3
                reward += 20.0
                info["log"] = "⚔️ Crafted Iron Pick"

        if self.steps >= HYPER_PARAMS["MAX_STEPS"]: done = True
        return self._get_obs(), reward, done, info

    def _min_dist_to_any(self):
        dists = [np.sum(np.abs(self.agent_pos - o["pos"])) for o in self.map_objects if o["pos"][0] != -1]
        return min(dists) if dists else 0

# --- WORLD MODEL HELPERS ---
def symlog(x): return torch.sign(x) * torch.log(torch.abs(x) + 1.0)
def symexp(x): return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)

# --- CELL 2: THE LIQUID DARK ARCHITECTURE (TURBO) ---
class LiquidCell(nn.Module):
    def __init__(self, input_size, hidden_size, delta_t=0.5):
        super().__init__()
        self.w_input = nn.Linear(input_size, hidden_size)
        self.w_state = nn.Linear(hidden_size, hidden_size)
        self.tau = nn.Parameter(torch.ones(1, hidden_size))
        self.A = nn.Parameter(torch.ones(1, hidden_size))

    def forward(self, x, h):
        sensory = torch.sigmoid(self.w_input(x) + self.w_state(h))
        h_new = h
        for _ in range(3):
            dh = (-(1.0 / torch.clamp(torch.exp(self.tau), min=1e-3) + sensory) * h_new + sensory * self.A)
            h_new = h_new + 0.5 * dh
        return h_new

class DarkHippocampus:
    def __init__(self, capacity=10000):
        self.data = []
        self.priorities = []
        self.capacity = capacity

    def push(self, exp, p):
        if len(self.data) >= self.capacity:
            self.data.pop(0)
            self.priorities.pop(0)
        self.data.append(exp)
        self.priorities.append(p + 1e-5)

    def sample(self, batch_size):
        p = np.array(self.priorities)
        p = p / p.sum()
        indices = np.random.choice(len(self.data), batch_size, p=p)
        return [self.data[i] for i in indices], indices

    def update_priorities(self, idxs, ps):
        for i, p in zip(idxs, ps):
            self.priorities[i] = p

class WorldModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Vector Projector instead of CNN
        self.encoder = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ELU(),
            nn.Linear(128, 256), nn.ELU()
        )
        self.liquid_core = LiquidCell(256 + 6, 256)
        self.reward_head = nn.Linear(256, 1)
        self.value_head = nn.Linear(256, 1)
        self.uncertainty_head = nn.Linear(256, 1)
        self.next_z_head = nn.Linear(256, 256)
        self.opt = optim.Adam(self.parameters(), lr=1e-3) # Faster LR

    def encode(self, obs):
        return self.encoder(obs)

    def dream_step(self, h, action):
        if isinstance(action, int) or len(action.shape) == 0:
            act_onehot = F.one_hot(torch.tensor(action, device=device).long(), 6).float().unsqueeze(0)
        else:
            act_onehot = F.one_hot(action.long(), 6).float()
        next_h = self.liquid_core(torch.cat([h, act_onehot], dim=1), h)
        return next_h, self.reward_head(next_h), self.value_head(next_h), torch.sigmoid(self.uncertainty_head(next_h)), self.next_z_head(next_h)

class LiquidDarkAgent(nn.Module):
    def __init__(self):
        super().__init__()
        self.wm = WorldModel().to(device)
        self.actor = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 6)).to(device)
        self.opt_actor = optim.Adam(self.actor.parameters(), lr=5e-4)
        self.hippocampus = DarkHippocampus()
        self.lambda_surprise = 2.0 # EXTREME CURIOSITY

    def plan_and_act(self, obs):
        h = self.wm.encode(torch.FloatTensor(obs).unsqueeze(0).to(device))
        best_a, max_score = 0, -1e9
        for a in range(6):
            curr_h, r_pred, v_pred, unc, _ = self.wm.dream_step(h, a)
            score = r_pred.item() + 0.95 * v_pred.item()
            horizon = 0
            while horizon < 20 and unc.item() < 0.3: # Turbo depth
                horizon += 1
                sim_a = torch.argmax(self.actor(curr_h), dim=1)
                curr_h, r_pred, v_pred, unc, _ = self.wm.dream_step(curr_h, sim_a)
                score += (0.95**horizon) * r_pred.item()
            if score > max_score: max_score, best_a = score, a
        return best_a

    def train_step(self):
        if len(self.hippocampus.data) < 64: return
        batch, idxs = self.hippocampus.sample(64)
        o, a, r, no, d = zip(*batch)
        o, no = torch.FloatTensor(np.array(o)).to(device), torch.FloatTensor(np.array(no)).to(device)
        a, r = torch.tensor(a).to(device), torch.FloatTensor(r).unsqueeze(1).to(device)

        z = self.wm.encode(o)
        with torch.no_grad(): target_z = self.wm.encode(no)
        h_next, r_p, v_p, u_p, nz_p = self.wm.dream_step(z, a)

        surprise = F.mse_loss(nz_p, target_z, reduction='none').mean(dim=1, keepdim=True)
        self.wm.opt.zero_grad()
        (F.mse_loss(nz_p, target_z) + F.mse_loss(r_p, symlog(r)) + F.mse_loss(u_p, surprise.detach())).backward()
        self.wm.opt.step()

        dist = torch.distributions.Categorical(logits=self.actor(h_next.detach()))
        act = dist.sample()
        adv = (symlog(r) + self.lambda_surprise * surprise.detach() - v_p.detach())
        loss_actor = -(dist.log_prob(act) * adv).mean()
        self.opt_actor.zero_grad()
        loss_actor.backward()
        self.opt_actor.step()
        self.hippocampus.update_priorities(idxs, surprise.flatten().detach().cpu().numpy())

# --- BENCHMARK ---
class StandardDQRN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ReLU(), nn.Linear(128, 6)).to(device)
    def act(self, obs):
        q = self.net(torch.FloatTensor(obs).to(device))
        return torch.argmax(q).item()

def run_benchmark():
    print("\n🚀 STARTING TURBO LIQUID DARK DUEL 🚀")
    env = TitanCraftEnv()
    dark, std = LiquidDarkAgent(), StandardDQRN()
    res = {"dark": [], "std": []}

    for ep in range(1, HYPER_PARAMS["EPISODES"] + 1):
        # DARK RUN
        o, total_r, d_found = env.reset(), 0, 0
        for t in range(HYPER_PARAMS["MAX_STEPS"]):
            a = dark.plan_and_act(o)
            if random.random() < 0.1: a = random.randint(0,5)
            no, r, done, info = env.step(a)

            with torch.no_grad():
                z, nz_t = dark.wm.encode(torch.FloatTensor(o).to(device)), dark.wm.encode(torch.FloatTensor(no).to(device))
                _, _, _, _, nz_p = dark.wm.dream_step(z.unsqueeze(0), a)
                s = F.mse_loss(nz_p.squeeze(), nz_t).item()

            dark.hippocampus.push((o, a, r, no, done), s)
            if t % 5 == 0: dark.train_step()
            total_r, o = total_r + r, no
            if env.inventory["diamond"] > 0: d_found = 1
            if done: break
        res["dark"].append(total_r)

        # STD RUN (Minimalist Baseline)
        o, r_std = env.reset(), 0
        for t in range(HYPER_PARAMS["MAX_STEPS"]):
            a = std.act(o)
            o, r, d, _ = env.step(a)
            r_std += r
            if d: break
        res["std"].append(r_std)

        if ep % 1 == 0 or d_found:
            print(f"Ep {ep}: Dark {total_r:.1f} (💎{d_found}) | Std {r_std:.1f}")

    print(f"\n🏆 FINAL: Std {np.mean(res['std']):.2f} | Dark {np.mean(res['dark']):.2f}")

if __name__ == "__main__":
    run_benchmark()


⚡ HARDWARE: Tesla T4

🚀 STARTING TURBO LIQUID DARK DUEL 🚀
Ep 1: Dark -4.6 (💎0) | Std -5.0


KeyboardInterrupt: 

# Version 2

In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import gymnasium as gym
from gymnasium import spaces
import cv2
import os
import tqdm
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚡ HARDWARE: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# --- CONFIGURATION ---
HYPER_PARAMS = {
    "MAX_STEPS": 200,
    "EPISODES": 50,
    "BATCH_SIZE": 64,
    "VECTOR_DIM": 9, # [x, y, vx, vy, sin_th, cos_th, dist, cur_x, cur_y]
}

# --- CELL 1: THE TITAN CRAFT ENVIRONMENT (TURBO) ---
class LiquidAbyssEnv(gym.Env):
    """
    A continuous control environment representing navigation in a chaotic vortex.
    Goal: Reach (0, 0).
    """
    def __init__(self):
        super(LiquidAbyssEnv, self).__init__()
        # Actions: [Thrust (0 to 1), Torque (-1 to 1)]
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-20, high=20, shape=(HYPER_PARAMS["VECTOR_DIM"],), dtype=np.float32)
        self.dt = 0.1
        self.intensity = 2.0

    def reset(self):
        # Start at a random position on a circle of radius 8
        angle = random.uniform(0, 2 * np.pi)
        self.pos = np.array([8.0 * np.cos(angle), 8.0 * np.sin(angle)])
        self.vel = np.array([0.0, 0.0])
        self.theta = random.uniform(0, 2 * np.pi)
        self.steps = 0
        return self._get_obs()

    def _get_obs(self):
        dist = np.linalg.norm(self.pos)
        cur_x = self.intensity * np.sin(self.pos[1] / 2.0)
        cur_y = self.intensity * np.cos(self.pos[0] / 2.0)
        obs = [
            self.pos[0] / 10.0, self.pos[1] / 10.0,
            self.vel[0] / 5.0, self.vel[1] / 5.0,
            np.sin(self.theta), np.cos(self.theta),
            dist / 10.0,
            cur_x / self.intensity, cur_y / self.intensity
        ]
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        self.steps += 1
        thrust = np.clip((action[0] + 1.0) / 2.0, 0.0, 1.0) # Map -1,1 to 0,1
        torque = action[1]

        # Current forces
        cur_x = self.intensity * np.sin(self.pos[1] / 2.0)
        cur_y = self.intensity * np.cos(self.pos[0] / 2.0)

        # Agent forces
        ax = thrust * np.cos(self.theta)
        ay = thrust * np.sin(self.theta)

        # Update physics
        self.vel[0] += (ax + cur_x) * self.dt
        self.vel[1] += (ay + cur_y) * self.dt
        self.pos += self.vel * self.dt
        self.theta += torque * self.dt * 2.0 # More rotation power

        dist = np.linalg.norm(self.pos)
        reward = -dist * 0.01 # Distance penalty
        reward -= 0.01 # Existential penalty

        # Reward for getting closer
        if dist < 0.5:
            reward += 10.0
            done = True
        elif self.steps >= HYPER_PARAMS["MAX_STEPS"]:
            done = True
        elif dist > 15.0: # Swept away
            reward -= 5.0
            done = True
        else:
            done = False

        return self._get_obs(), reward, done, {}
# --- WORLD MODEL HELPERS ---
def symlog(x): return torch.sign(x) * torch.log(torch.abs(x) + 1.0)
def symexp(x): return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)

# --- CELL 2: THE LIQUID DARK ARCHITECTURE (TURBO) ---
class LiquidCell(nn.Module):
    def __init__(self, input_size, hidden_size, delta_t=0.5):
        super().__init__()
        self.w_input = nn.Linear(input_size, hidden_size)
        self.w_state = nn.Linear(hidden_size, hidden_size)
        self.tau = nn.Parameter(torch.ones(1, hidden_size))
        self.A = nn.Parameter(torch.ones(1, hidden_size))

    def forward(self, x, h):
        sensory = torch.sigmoid(self.w_input(x) + self.w_state(h))
        h_new = h
        for _ in range(3):
            dh = (-(1.0 / torch.clamp(torch.exp(self.tau), min=1e-3) + sensory) * h_new + sensory * self.A)
            h_new = h_new + 0.5 * dh
        return h_new

class DarkHippocampus:
    def __init__(self, capacity=10000):
        self.data = []
        self.priorities = []
        self.capacity = capacity

    def push(self, exp, p):
        if len(self.data) >= self.capacity:
            self.data.pop(0)
            self.priorities.pop(0)
        self.data.append(exp)
        self.priorities.append(p + 1e-5)

    def sample(self, batch_size):
        p = np.array(self.priorities)
        p = p / p.sum()
        indices = np.random.choice(len(self.data), batch_size, p=p)
        return [self.data[i] for i in indices], indices

    def update_priorities(self, idxs, ps):
        for i, p in zip(idxs, ps):
            self.priorities[i] = p

class WorldModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Vector Projector instead of CNN
        self.encoder = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ELU(),
            nn.Linear(128, 256), nn.ELU()
        )
        self.liquid_core = LiquidCell(256 + 2, 256) # 256 state + 2 action
        self.reward_head = nn.Linear(256, 1)
        self.value_head = nn.Linear(256, 1)
        self.uncertainty_head = nn.Linear(256, 1)
        self.next_z_head = nn.Linear(256, 256)
        self.opt = optim.Adam(self.parameters(), lr=1e-3)

    def encode(self, obs):
        return self.encoder(obs)

    def dream_step(self, h, action):
        # Action is [batch, 2]
        if len(action.shape) == 1:
            action = action.unsqueeze(0)
        next_h = self.liquid_core(torch.cat([h, action], dim=1), h)
        return next_h, self.reward_head(next_h), self.value_head(next_h), torch.sigmoid(self.uncertainty_head(next_h)), self.next_z_head(next_h)

class LiquidDarkAgent(nn.Module):
    def __init__(self):
        super().__init__()
        self.wm = WorldModel().to(device)
        # Actor outputs [mu_thrust, mu_torque, log_std_thrust, log_std_torque]
        self.actor = nn.Sequential(
            nn.Linear(256, 128), nn.ELU(),
            nn.Linear(128, 4)
        ).to(device)
        self.opt_actor = optim.Adam(self.actor.parameters(), lr=5e-4)
        self.hippocampus = DarkHippocampus()
        self.lambda_surprise = 2.0
        self.num_candidates = 8 # Sample 8 trajectories

    def get_action_dist(self, h):
        out = self.actor(h)
        mu, log_std = out[:, :2], out[:, 2:]
        std = torch.exp(torch.clamp(log_std, -5, 2))
        return torch.distributions.Normal(mu, std)

    def plan_and_act(self, obs):
        h = self.wm.encode(torch.FloatTensor(obs).unsqueeze(0).to(device))

        # CEM-like selection in imagination
        best_a, max_score = None, -1e9

        # Try a few random actions + the actor's mu
        candidates = []
        with torch.no_grad():
            dist = self.get_action_dist(h)
            candidates.append(dist.mean) # The "Expert" suggestion
            for _ in range(self.num_candidates - 1):
                candidates.append(dist.sample())

        for a_cand in candidates:
            curr_h, r_pred, v_pred, unc, _ = self.wm.dream_step(h, a_cand)
            score = r_pred.item() + 0.95 * v_pred.item()

            # Deeper dreaming if confident
            horizon = 0
            while horizon < 10 and unc.item() < 0.3:
                horizon += 1
                sim_a = self.get_action_dist(curr_h).sample()
                curr_h, r_pred, v_pred, unc, _ = self.wm.dream_step(curr_h, sim_a)
                score += (0.95**horizon) * r_pred.item()

            if score > max_score:
                max_score = score
                best_a = a_cand

        return best_a.squeeze().cpu().numpy()

    def train_step(self):
        if len(self.hippocampus.data) < 64: return
        batch, idxs = self.hippocampus.sample(64)
        o, a, r, no, d = zip(*batch)
        o, no = torch.FloatTensor(np.array(o)).to(device), torch.FloatTensor(np.array(no)).to(device)
        a, r = torch.FloatTensor(np.array(a)).to(device), torch.FloatTensor(r).unsqueeze(1).to(device)

        z = self.wm.encode(o)
        with torch.no_grad(): target_z = self.wm.encode(no)
        h_next, r_p, v_p, u_p, nz_p = self.wm.dream_step(z, a)

        surprise = F.mse_loss(nz_p, target_z, reduction='none').mean(dim=1, keepdim=True)
        self.wm.opt.zero_grad()
        (F.mse_loss(nz_p, target_z) + F.mse_loss(r_p, symlog(r)) + F.mse_loss(u_p, surprise.detach())).backward()
        self.wm.opt.step()

        dist = self.get_action_dist(h_next.detach())
        act = dist.sample()
        # Intrinsic Reward: Surprise!
        adv = (symlog(r) + self.lambda_surprise * surprise.detach() - v_p.detach())
        loss_actor = -(dist.log_prob(act).sum(dim=1, keepdim=True) * adv).mean()

        self.opt_actor.zero_grad()
        loss_actor.backward()
        self.opt_actor.step()
        self.hippocampus.update_priorities(idxs, surprise.flatten().detach().cpu().numpy())

# --- BENCHMARK ---
class StandardMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ReLU(),
            nn.Linear(128, 2)
        ).to(device)
    def act(self, obs):
        with torch.no_grad():
            a = self.net(torch.FloatTensor(obs).to(device))
        return np.clip(a.cpu().numpy(), -1.0, 1.0)

def run_benchmark():
    print("\n🚀 STARTING LIQUID ABYSS CHALLENGE 🚀")
    env = LiquidAbyssEnv()
    dark, std = LiquidDarkAgent(), StandardMLP()
    res = {"dark": [], "std": []}

    for ep in range(1, HYPER_PARAMS["EPISODES"] + 1):
        # DARK RUN
        o, total_r, success = env.reset(), 0, 0
        for t in range(HYPER_PARAMS["MAX_STEPS"]):
            a = dark.plan_and_act(o)
            # Add some exploration noise to data collection
            noise = np.random.normal(0, 0.1, size=2)
            no, r, done, info = env.step(a + noise)

            with torch.no_grad():
                z = dark.wm.encode(torch.FloatTensor(o).to(device)).unsqueeze(0)
                nz_t = dark.wm.encode(torch.FloatTensor(no).to(device)).unsqueeze(0)
                _, _, _, _, nz_p = dark.wm.dream_step(z, torch.FloatTensor(a).to(device).unsqueeze(0))
                s = F.mse_loss(nz_p, nz_t).item()

            dark.hippocampus.push((o, a, r, no, done), s)
            if t % 5 == 0: dark.train_step()
            total_r, o = total_r + r, no
            if done:
                if np.linalg.norm(env.pos) < 0.5: success = 1
                break
        res["dark"].append(total_r)

        # STD RUN (Random/Simple Baseline)
        o, r_std = env.reset(), 0
        for t in range(HYPER_PARAMS["MAX_STEPS"]):
            a = std.act(o)
            o, r, d, _ = env.step(a)
            r_std += r
            if d: break
        res["std"].append(r_std)

        if ep % 5 == 0 or success:
            print(f"Ep {ep}: Dark {total_r:.1f} ({'🎯' if success else '🌊'}) | Std {r_std:.1f}")

    print(f"\n🏆 FINAL AVG: Std {np.mean(res['std']):.2f} | Dark {np.mean(res['dark']):.2f}")

if __name__ == "__main__":
    run_benchmark()


# Version 3

In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import gymnasium as gym
from gymnasium import spaces
import cv2
import os
import tqdm
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚡ HARDWARE: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# --- CONFIGURATION ---
SEED = 42
HYPER_PARAMS = {
    "MAX_STEPS": 200,
    "EPISODES_PER_LEVEL": 10,
    "BATCH_SIZE": 16, # Sequence batch size
    "SEQ_LEN": 16,    # Length of imagination/training sequence
    "HORIZON": 15,    # Planning horizon
    "VECTOR_DIM": 10,
    "DIAMOND_REWARD": 50.0,
    "MINE_DISTANCE": 0.8,
}

LEVEL_DEFS = {
    1: {"name": "Abyssal Grass", "intensity": 0.5, "diamonds": 15},
    2: {"name": "Vortex Woods", "intensity": 1.2, "diamonds": 7},
    3: {"name": "Event Horizon", "intensity": 2.5, "diamonds": 3},
}

# --- CELL 1: THE TITAN CRAFT ENVIRONMENT (TURBO) ---
class LiquidAbyssEnv(gym.Env):
    """
    A continuous control environment representing navigation in a chaotic vortex.
    Goal: Reach (0, 0).
    """
    def __init__(self):
        super(LiquidAbyssEnv, self).__init__()
        # Actions: [Thrust (0 to 1), Torque (-1 to 1)]
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-20, high=20, shape=(HYPER_PARAMS["VECTOR_DIM"],), dtype=np.float32)
        self.dt = 0.1
        self.intensity = 1.0 # Start easy
        self.difficulty_cycle = 5
        self.diamonds = []

    def reset(self, level=1, seed=None):
        # Deterministic seeding for fairness
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        # Start at a random position on a circle of radius 8
        angle = random.uniform(0, 2 * np.pi)
        self.pos = np.array([8.0 * np.cos(angle), 8.0 * np.sin(angle)])
        self.vel = np.array([0.0, 0.0])
        self.theta = random.uniform(0, 2 * np.pi)
        self.steps = 0

        # Level-based difficulty
        level_cfg = LEVEL_DEFS.get(level, LEVEL_DEFS[1])
        self.intensity = level_cfg["intensity"]
        self.diamonds = [np.array([random.uniform(-12, 12), random.uniform(-12, 12)]) for _ in range(level_cfg["diamonds"])]

        return self._get_obs()

    def _get_obs(self):
        dist = np.linalg.norm(self.pos)
        cur_x = self.intensity * np.sin(self.pos[1] / 2.0)
        cur_y = self.intensity * np.cos(self.pos[0] / 2.0)

        # Proximity to nearest diamond
        if self.diamonds:
            diamond_dists = [np.linalg.norm(self.pos - d) for d in self.diamonds]
            min_diamond_dist = min(diamond_dists)
        else:
            min_diamond_dist = 20.0

        obs = [
            self.pos[0] / 10.0, self.pos[1] / 10.0,
            self.vel[0] / 5.0, self.vel[1] / 5.0,
            np.sin(self.theta), np.cos(self.theta),
            dist / 10.0,
            cur_x / self.intensity, cur_y / self.intensity,
            min_diamond_dist / 10.0
        ]
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        self.steps += 1
        thrust = np.clip((action[0] + 1.0) / 2.0, 0.0, 1.0) # Map -1,1 to 0,1
        torque = action[1]

        # Current forces
        cur_x = self.intensity * np.sin(self.pos[1] / 2.0)
        cur_y = self.intensity * np.cos(self.pos[0] / 2.0)

        # Agent forces
        ax = thrust * np.cos(self.theta)
        ay = thrust * np.sin(self.theta)

        # Update physics
        self.vel[0] += (ax + cur_x) * self.dt
        self.vel[1] += (ay + cur_y) * self.dt
        self.pos += self.vel * self.dt
        self.theta += torque * self.dt * 2.0 # More rotation power

        dist = np.linalg.norm(self.pos)
        reward = -dist * 0.01 # Distance penalty
        reward -= 0.01 # Existential penalty

        # Diamond Mining Logic
        mined_indices = []
        self.last_mined_count = 0
        for i, diamond_pos in enumerate(self.diamonds):
            if np.linalg.norm(self.pos - diamond_pos) < HYPER_PARAMS["MINE_DISTANCE"]:
                reward += HYPER_PARAMS["DIAMOND_REWARD"]
                mined_indices.append(i)

        self.last_mined_count = len(mined_indices)
        # Remove mined diamonds
        for i in sorted(mined_indices, reverse=True):
            self.diamonds.pop(i)

        # Reward for getting closer to origin (optional goal)
        if dist < 0.5:
            reward += 10.0
            done = True
        elif self.steps >= HYPER_PARAMS["MAX_STEPS"]:
            done = True
        elif dist > 20.0: # Swept away (increased range for easier mode)
            reward -= 5.0
            done = True
        else:
            done = False

        return self._get_obs(), reward, done, {}
# --- WORLD MODEL HELPERS ---
def symlog(x): return torch.sign(x) * torch.log(torch.abs(x) + 1.0)
def symexp(x): return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)

# --- CELL 2: THE LIQUID DARK ARCHITECTURE (DREAMER V4) ---
class LiquidCell(nn.Module):
    """
    Liquid Time-Constant (LTC) Cell.
    Adapts its time-constant (tau) based on input context.
    """
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.w_input = nn.Linear(input_size, hidden_size)
        self.w_tau = nn.Linear(input_size + hidden_size, hidden_size)
        self.w_state = nn.Linear(hidden_size, hidden_size)
        self.A = nn.Parameter(torch.ones(1, hidden_size))

    def forward(self, x, h):
        # Continuous-time dynamics
        sensory = torch.tanh(self.w_input(x) + self.w_state(h))

        # Liquid Time Constant: Tau depends on input + state
        tau_gate = torch.sigmoid(self.w_tau(torch.cat([x, h], dim=-1)))
        tau = 1.0 + 5.0 * tau_gate # Range [1.0, 6.0]

        # ODE Solver (Euler Step with small dt)
        h_new = h
        dt = 0.1
        for _ in range(3): # solver steps
            dh = (-(h_new - self.A * sensory) / tau)
            h_new = h_new + dt * dh
        return h_new

class SequenceBuffer:
    """Stores full episodes for sequence learning."""
    def __init__(self, capacity=2000):
        self.episodes = []
        self.capacity = capacity

    def push_episode(self, episode_data):
        # episode_data: list of (obs, action, reward, done)
        if len(self.episodes) >= self.capacity:
            self.episodes.pop(0)
        self.episodes.append(episode_data)

    def sample_sequence(self, batch_size, seq_len):
        if not self.episodes: return None

        # Sample random episodes, then random chunks
        obs_b, act_b, rew_b, term_b = [], [], [], []

        for _ in range(batch_size):
            ep = random.choice(self.episodes)
            if len(ep) < seq_len + 1:
                # Pad if too short
                start_idx = 0
                segment = ep + [ep[-1]] * (seq_len + 1 - len(ep))
            else:
                start_idx = random.randint(0, len(ep) - seq_len - 1)
                segment = ep[start_idx : start_idx + seq_len + 1]

            # Unzip
            o, a, r, d = zip(*segment)
            obs_b.append(np.array(o))
            act_b.append(np.array(a))
            rew_b.append(np.array(r))
            term_b.append(np.array(d))

        return (torch.FloatTensor(np.array(obs_b)).to(device),
                torch.FloatTensor(np.array(act_b)).to(device),
                torch.FloatTensor(np.array(rew_b)).to(device),
                torch.FloatTensor(np.array(term_b)).to(device))

class WorldModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder: Projects Observation to State Encodings
        self.encoder = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ELU(),
            nn.Linear(128, 256), nn.ELU()
        )

        # RSSM-lite: Deterministic Liquid Path
        self.core = LiquidCell(256 + 2, 256)

        # Heads
        self.reward_head = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 1))
        self.value_head = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 1))
        self.cont_head = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 1)) # Continuation (Success/Survivor)

        self.opt = optim.Adam(self.parameters(), lr=3e-4)

    def forward_seq(self, obs_seq, act_seq, init_h=None):
        # Connects time steps
        b, t, _ = obs_seq.shape
        embedded = self.encoder(obs_seq)

        states = []
        if init_h is None: init_h = torch.zeros(b, 256).to(device)
        h = init_h

        for i in range(t):
            # Input: Embedding + Previous Action
            h = self.core(torch.cat([embedded[:, i], act_seq[:, i]], dim=-1), h)
            states.append(h)

        states = torch.stack(states, dim=1)
        return states

class LiquidDarkAgent(nn.Module):
    def __init__(self):
        super().__init__()
        self.wm = WorldModel().to(device)
        self.actor = nn.Sequential(
            nn.Linear(256, 128), nn.ELU(),
            nn.Linear(128, 2), nn.Tanh() # Deterministic Actor for simplicity & speed in Dreamer
        ).to(device)

        self.opt_actor = optim.Adam(self.actor.parameters(), lr=1e-4)
        self.buffer = SequenceBuffer()
        self.h_state = None

    def reset_state(self):
        self.h_state = torch.zeros(1, 256).to(device)

    def plan_and_act(self, obs):
        # Inference: Direct Policy with Liquid State
        with torch.no_grad():
            if self.h_state is None: self.reset_state()

            embedded = self.wm.encoder(torch.FloatTensor(obs).unsqueeze(0).to(device))

            # Dynamics Update (Perception)
            # We assume 'zero' previous action for perception update in this simplified loop
            dummy_action = torch.zeros(1, 2).to(device)
            self.h_state = self.wm.core(torch.cat([embedded, dummy_action], dim=-1), self.h_state)

            # Action
            action = self.actor(self.h_state)

        return action.squeeze().cpu().numpy()

    def imagine(self, start_states, horizon=15):
        # Dreamer Rollout
        states = [start_states]
        actions = []

        curr_h = start_states
        for _ in range(horizon):
            a = self.actor(curr_h.detach())
            a_noisy = a + torch.randn_like(a) * 0.1

            # Imagine dynamics (Thinking)
            dummy_embed = torch.zeros(curr_h.shape[0], 256).to(device)
            curr_h = self.wm.core(torch.cat([dummy_embed, a_noisy], dim=-1), curr_h)

            states.append(curr_h)
            actions.append(a)

        return torch.stack(states, dim=1), torch.stack(actions, dim=1)

    def train_step(self):
        batch = self.buffer.sample_sequence(HYPER_PARAMS["BATCH_SIZE"], HYPER_PARAMS["SEQ_LEN"])
        if batch is None: return
        obs, act, rew, term = batch

        # 1. Train World Model
        model_states = self.wm.forward_seq(obs, act) # (B, T, 256)

        pred_rews = self.wm.reward_head(model_states).squeeze(-1)
        pred_vals = self.wm.value_head(model_states).squeeze(-1)

        target_rews = symlog(rew)
        loss_reward = F.mse_loss(pred_rews, target_rews)

        self.wm.opt.zero_grad()
        loss_reward.backward()
        self.wm.opt.step()

        # 2. Train Actor (Deep Planning)
        start_states = model_states[:, -1].detach()
        dream_states, dream_acts = self.imagine(start_states, horizon=HYPER_PARAMS["HORIZON"])

        dream_rews = self.wm.reward_head(dream_states).squeeze(-1)
        dream_vals = self.wm.value_head(dream_states).squeeze(-1)

        # Maximize Dreamed Value + Reward
        loss_actor = -(dream_vals.mean() + dream_rews.mean())

        self.opt_actor.zero_grad()
        loss_actor.backward()
        self.opt_actor.step()

# --- BENCHMARK ---
class StandardMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ReLU(),
            nn.Linear(128, 2)
        ).to(device)
    def act(self, obs):
        with torch.no_grad():
            a = self.net(torch.FloatTensor(obs).to(device))
        return np.clip(a.cpu().numpy(), -1.0, 1.0)

def run_benchmark():
    print("\n" + "="*60)
    print("🚀 STARTING UPGRADED MULTI-LEVEL CHALLENGE 🚀")
    print(f"⚡ HARDWARE: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    print("="*60)

    env = LiquidAbyssEnv()
    dark, std = LiquidDarkAgent(), StandardMLP()

    overall_res = {"dark": {"rewards": [], "diamonds": [], "success": []},
                   "std": {"rewards": [], "diamonds": [], "success": []}}

    for lvl_idx in [1, 2, 3]:
        lvl_cfg = LEVEL_DEFS[lvl_idx]
        print(f"\n🏔️ LEVEL {lvl_idx}: {lvl_cfg['name'].upper()}")
        print(f"   [Intensity: {lvl_cfg['intensity']} | Resources: {lvl_cfg['diamonds']} Diamonds]")
        print("-" * 60)

        lvl_rewards_dark, lvl_diamonds_dark, lvl_success_dark = [], [], []
        lvl_rewards_std, lvl_diamonds_std, lvl_success_std = [], [], []

        for ep in range(1, HYPER_PARAMS["EPISODES_PER_LEVEL"] + 1):
            shared_seed = SEED + lvl_idx * 100 + ep

            # --- DARK RUN ---
            o, total_r, success, diamonds_mined = env.reset(level=lvl_idx, seed=shared_seed), 0, 0, 0
            steps, total_dist = 0, 0
            dark.reset_state()
            episode_history = [] # Buffer for sequence learning

            for t in range(HYPER_PARAMS["MAX_STEPS"]):
                a = dark.plan_and_act(o)
                no, r, done, _ = env.step(a)

                # Store transition
                episode_history.append((o, a, r, done))

                # Train frequently! (Dreamer style: 1 step = 1 train)
                if t > HYPER_PARAMS["SEQ_LEN"]:
                    dark.train_step()

                total_r += r
                diamonds_mined += env.last_mined_count
                total_dist += np.linalg.norm(env.pos)
                steps += 1
                o = no
                if done:
                    if np.linalg.norm(env.pos) < 0.5: success = 1
                    break

            # Push episode to replay buffer
            dark.buffer.push_episode(episode_history)

            lvl_rewards_dark.append(total_r)
            lvl_diamonds_dark.append(diamonds_mined)
            lvl_success_dark.append(success)
            avg_dist_dark = total_dist / max(1, steps)

            # --- STD RUN ---
            o, total_r_std, success_std, diamonds_mined_std = env.reset(level=lvl_idx, seed=shared_seed), 0, 0, 0
            steps_std, total_dist_std = 0, 0
            for t in range(HYPER_PARAMS["MAX_STEPS"]):
                a = std.act(o)
                no, r, done, _ = env.step(a)
                total_r_std += r
                diamonds_mined_std += env.last_mined_count
                total_dist_std += np.linalg.norm(env.pos)
                steps_std += 1
                o = no
                if done:
                    if np.linalg.norm(env.pos) < 0.5: success_std = 1
                    break
            lvl_rewards_std.append(total_r_std)
            lvl_diamonds_std.append(diamonds_mined_std)
            lvl_success_std.append(success_std)
            avg_dist_std = total_dist_std / max(1, steps_std)

            # --- DETAILED EPISODE OUTPUT ---
            d_icon = "💎" * (diamonds_mined // 1)
            s_icon = "💎" * (diamonds_mined_std // 1)
            print(f"Ep {ep} | DARK: {total_r:6.1f} [{d_icon:10}] {'🎯' if success else '🌊'} (Dist: {avg_dist_dark:4.1f})")
            print(f"     | STD : {total_r_std:6.1f} [{s_icon:10}] {'🎯' if success_std else '🌊'} (Dist: {avg_dist_std:4.1f})")
            print("-" * 35)

        # LEVEL SUMMARY
        print(f"\n📊 SUMMARY - LEVEL {lvl_idx}: {lvl_cfg['name']}")
        def print_stats(name, rewards, diamonds, success):
            print(f"   {name:5}: Avg R: {np.mean(rewards):7.1f} | Tot Diamonds: {sum(diamonds):2} | Success: {sum(success)}/{len(success)}")

        print_stats("DARK", lvl_rewards_dark, lvl_diamonds_dark, lvl_success_dark)
        print_stats("STD", lvl_rewards_std, lvl_diamonds_std, lvl_success_std)
        print("-" * 60)

        # Update overall
        overall_res["dark"]["rewards"].extend(lvl_rewards_dark)
        overall_res["dark"]["diamonds"].extend(lvl_diamonds_dark)
        overall_res["dark"]["success"].extend(lvl_success_dark)
        overall_res["std"]["rewards"].extend(lvl_rewards_std)
        overall_res["std"]["diamonds"].extend(lvl_diamonds_std)
        overall_res["std"]["success"].extend(lvl_success_std)

    # FINAL GRAND SUMMARY
    print("\n" + "🏆" * 20)
    print("      FINAL BENCHMARK CHAMPION")
    print("🏆" * 20)
    dark_avg = np.mean(overall_res["dark"]["rewards"])
    std_avg = np.mean(overall_res["std"]["rewards"])
    winner = "LIQUID DARK AGENT" if dark_avg > std_avg else "STANDARD MLP"

    print(f"   RANK #1: {winner}")
    print(f"   DARK | Avg Score: {dark_avg:7.2f} | Diamonds: {sum(overall_res['dark']['diamonds']):>3} | Wins: {sum(overall_res['dark']['success'])}")
    print(f"   STD  | Avg Score: {std_avg:7.2f} | Diamonds: {sum(overall_res['std']['diamonds']):>3} | Wins: {sum(overall_res['std']['success'])}")
    print("="*60 + "\n")

if __name__ == "__main__":
    run_benchmark()


⚡ HARDWARE: Tesla T4

🚀 STARTING UPGRADED MULTI-LEVEL CHALLENGE 🚀
⚡ HARDWARE: Tesla T4

🏔️ LEVEL 1: ABYSSAL GRASS
   [Intensity: 0.5 | Resources: 15 Diamonds]
------------------------------------------------------------
Ep 1 | DARK:   37.6 [💎         ] 🌊 (Dist: 12.0)
     | STD :  -12.5 [          ] 🌊 (Dist: 11.9)
-----------------------------------
Ep 2 | DARK:   35.5 [💎         ] 🌊 (Dist: 10.7)
     | STD :   77.7 [💎💎        ] 🌊 (Dist: 10.2)
-----------------------------------
Ep 3 | DARK:  -11.8 [          ] 🌊 (Dist: 12.3)
     | STD :  -12.0 [          ] 🌊 (Dist: 12.2)
-----------------------------------
Ep 4 | DARK:  -12.5 [          ] 🌊 (Dist:  7.3)
     | STD :   27.5 [💎         ] 🌊 (Dist: 10.3)
-----------------------------------
Ep 5 | DARK:   22.1 [💎         ] 🌊 (Dist: 13.0)
     | STD :   32.2 [💎         ] 🌊 (Dist: 10.7)
-----------------------------------
Ep 6 | DARK:   80.4 [💎💎        ] 🌊 (Dist:  8.7)
     | STD :  -14.7 [          ] 🌊 (Dist: 10.1)
------------------------

# Version 4

In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import gymnasium as gym
from gymnasium import spaces
import cv2
import os
import tqdm
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚡ HARDWARE: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# --- CONFIGURATION ---
SEED = 42
HYPER_PARAMS = {
    "MAX_STEPS": 200,
    "EPISODES_PER_LEVEL": 10,
    "BATCH_SIZE": 32,
    "SEQ_LEN": 16,
    "HORIZON": 50,        # DEEP DREAMING: Long horizon for long-term planning
    "VECTOR_DIM": 10,
    "DIAMOND_REWARD": 100.0, # Massive reward for reaching sparse targets
    "MINE_DISTANCE": 0.8,
    "WARMUP_EPISODES": 5,
    "TRAIN_PER_STEP": 2,
    "GAMMA": 0.99,
    "LAMBDA": 0.95,
    "EXPLORE_NOISE": 0.1,
    "CURIOSITY_WEIGHT": 0.05 # Weight for surprise-based exploration
}

LEVEL_DEFS = {
    1: {"name": "Abyssal Grass", "intensity": 0.5, "diamonds": 15},
    2: {"name": "Vortex Woods", "intensity": 1.2, "diamonds": 7},
    3: {"name": "Event Horizon", "intensity": 2.5, "diamonds": 3},
}

# --- CELL 1: THE TITAN CRAFT ENVIRONMENT (TURBO) ---
class LiquidAbyssEnv(gym.Env):
    """
    A continuous control environment representing navigation in a chaotic vortex.
    Goal: Reach (0, 0).
    """
    def __init__(self):
        super(LiquidAbyssEnv, self).__init__()
        # Actions: [Thrust (0 to 1), Torque (-1 to 1)]
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-20, high=20, shape=(HYPER_PARAMS["VECTOR_DIM"],), dtype=np.float32)
        self.dt = 0.1
        self.intensity = 1.0 # Start easy
        self.difficulty_cycle = 5
        self.diamonds = []

    def reset(self, level=1, seed=None):
        # Deterministic seeding for fairness
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        # Start at a random position on a circle of radius 8
        angle = random.uniform(0, 2 * np.pi)
        self.pos = np.array([8.0 * np.cos(angle), 8.0 * np.sin(angle)])
        self.vel = np.array([0.0, 0.0])
        self.theta = random.uniform(0, 2 * np.pi)
        self.steps = 0

        # Level-based difficulty
        level_cfg = LEVEL_DEFS.get(level, LEVEL_DEFS[1])
        self.intensity = level_cfg["intensity"]

        # STRATEGIC PLACEMENT: Diamonds spawn far away (10 to 14 units)
        # This puts them in the high-turbulence zone where luck fails.
        self.diamonds = []
        for _ in range(level_cfg["diamonds"]):
            angle = random.uniform(0, 2 * np.pi)
            dist = random.uniform(10.0, 14.0)
            self.diamonds.append(np.array([dist * np.cos(angle), dist * np.sin(angle)]))

        return self._get_obs()

    def _get_obs(self):
        dist = np.linalg.norm(self.pos)
        cur_x = self.intensity * np.sin(self.pos[1] / 2.0)
        cur_y = self.intensity * np.cos(self.pos[0] / 2.0)

        # Proximity to nearest diamond
        if self.diamonds:
            diamond_dists = [np.linalg.norm(self.pos - d) for d in self.diamonds]
            min_diamond_dist = min(diamond_dists)
        else:
            min_diamond_dist = 20.0

        obs = [
            self.pos[0] / 10.0, self.pos[1] / 10.0,
            self.vel[0] / 5.0, self.vel[1] / 5.0,
            np.sin(self.theta), np.cos(self.theta),
            dist / 10.0,
            cur_x / self.intensity, cur_y / self.intensity,
            min_diamond_dist / 10.0
        ]
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        self.steps += 1
        thrust = np.clip((action[0] + 1.0) / 2.0, 0.0, 1.0)
        torque = action[1]

        # CURRENT INTENSITY: Multiply by 2.0 for higher chaos
        cur_x = (self.intensity * 2.0) * np.sin(self.pos[1] / 2.0)
        cur_y = (self.intensity * 2.0) * np.cos(self.pos[0] / 2.0)

        # Agent forces
        ax = thrust * np.cos(self.theta)
        ay = thrust * np.sin(self.theta)

        # Update physics
        self.vel[0] += (ax + cur_x) * self.dt
        self.vel[1] += (ay + cur_y) * self.dt
        self.pos += self.vel * self.dt
        self.theta += torque * self.dt * 2.0

        dist = np.linalg.norm(self.pos)

        # SPARSE REWARD: No distance penalty anymore.
        # Only existential cost to encourage speed.
        reward = -0.05

        # Diamond Mining (Checkpoints)
        mined_indices = []
        self.last_mined_count = 0
        for i, diamond_pos in enumerate(self.diamonds):
            if np.linalg.norm(self.pos - diamond_pos) < HYPER_PARAMS["MINE_DISTANCE"]:
                reward += HYPER_PARAMS["DIAMOND_REWARD"]
                mined_indices.append(i)

        self.last_mined_count = len(mined_indices)
        for i in sorted(mined_indices, reverse=True):
            self.diamonds.pop(i)

        # SUCCESS CRITERIA: Reach center
        if dist < 0.6:
            reward += 500.0 # Massive success reward
            done = True
        elif self.steps >= HYPER_PARAMS["MAX_STEPS"]:
            done = True
        elif dist > 15.0: # Stricter out-of-bounds (15 instead of 20)
            reward -= 50.0 # Heavy failure penalty
            done = True
        else:
            done = False

        return self._get_obs(), reward, done, {}
# --- WORLD MODEL HELPERS ---
def symlog(x): return torch.sign(x) * torch.log(torch.abs(x) + 1.0)
def symexp(x): return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)

# --- CELL 2: THE LIQUID DARK ARCHITECTURE (DREAMER V4) ---
class LiquidCell(nn.Module):
    """
    Liquid Time-Constant (LTC) Cell.
    Adapts its time-constant (tau) based on input context.
    """
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.w_input = nn.Linear(input_size, hidden_size)
        self.w_tau = nn.Linear(input_size + hidden_size, hidden_size)
        self.w_state = nn.Linear(hidden_size, hidden_size)
        self.A = nn.Parameter(torch.ones(1, hidden_size))

    def forward(self, x, h):
        # Continuous-time dynamics
        sensory = torch.tanh(self.w_input(x) + self.w_state(h))

        # Liquid Time Constant: Tau depends on input + state
        tau_gate = torch.sigmoid(self.w_tau(torch.cat([x, h], dim=-1)))
        tau = 1.0 + 5.0 * tau_gate # Range [1.0, 6.0]

        # ODE Solver (Euler Step with small dt)
        h_new = h
        dt = 0.1
        for _ in range(3): # solver steps
            dh = (-(h_new - self.A * sensory) / tau)
            h_new = h_new + dt * dh
        return h_new

class SequenceBuffer:
    """Stores full episodes for sequence learning."""
    def __init__(self, capacity=2000):
        self.episodes = []
        self.capacity = capacity

    def push_episode(self, episode_data):
        # episode_data: list of (obs, action, reward, done)
        if len(self.episodes) >= self.capacity:
            self.episodes.pop(0)
        self.episodes.append(episode_data)

    def sample_sequence(self, batch_size, seq_len):
        if not self.episodes: return None

        # Sample random episodes, then random chunks
        obs_b, act_b, rew_b, term_b = [], [], [], []

        for _ in range(batch_size):
            ep = random.choice(self.episodes)
            if len(ep) < seq_len + 1:
                # Pad if too short
                start_idx = 0
                segment = ep + [ep[-1]] * (seq_len + 1 - len(ep))
            else:
                start_idx = random.randint(0, len(ep) - seq_len - 1)
                segment = ep[start_idx : start_idx + seq_len + 1]

            # Unzip
            o, a, r, d = zip(*segment)
            obs_b.append(np.array(o))
            act_b.append(np.array(a))
            rew_b.append(np.array(r))
            term_b.append(np.array(d))

        return (torch.FloatTensor(np.array(obs_b)).to(device),
                torch.FloatTensor(np.array(act_b)).to(device),
                torch.FloatTensor(np.array(rew_b)).to(device),
                torch.FloatTensor(np.array(term_b)).to(device))

class WorldModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder: Projects Observation to State Encodings
        self.encoder = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ELU(),
            nn.Linear(128, 256), nn.ELU()
        )

        # RSSM-lite: Deterministic Liquid Path
        self.core = LiquidCell(256 + 2, 256)

        # Heads
        self.reward_head = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 1))
        self.value_head = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 1))
        self.cont_head = nn.Sequential(nn.Linear(256, 128), nn.ELU(), nn.Linear(128, 1), nn.Sigmoid())

        # Decoder: Reconstruct observation from state (helps learn better representations)
        self.decoder = nn.Sequential(
            nn.Linear(256, 128), nn.ELU(),
            nn.Linear(128, HYPER_PARAMS["VECTOR_DIM"])
        )

        # Dynamics predictor: Predict next state embedding for imagination
        self.dynamics = nn.Sequential(
            nn.Linear(256 + 2, 256), nn.ELU(),
            nn.Linear(256, 256)
        )

        self.opt = optim.Adam(self.parameters(), lr=3e-4)

    def forward_seq(self, obs_seq, act_seq, init_h=None):
        # Connects time steps
        b, t, _ = obs_seq.shape
        embedded = self.encoder(obs_seq)

        states = []
        if init_h is None: init_h = torch.zeros(b, 256).to(device)
        h = init_h

        for i in range(t):
            # Input: Embedding + Previous Action
            h = self.core(torch.cat([embedded[:, i], act_seq[:, i]], dim=-1), h)
            states.append(h)

        states = torch.stack(states, dim=1)
        return states, embedded  # Also return embeddings for dynamics training

    def imagine_step(self, h, action):
        """Pure imagination step using learned dynamics."""
        dyn_input = torch.cat([h, action], dim=-1)
        h_next = self.dynamics(dyn_input)
        # Mix with liquid core for temporal coherence
        h_next = self.core(torch.cat([h_next, action], dim=-1), h)
        return h_next

class LiquidDarkAgent(nn.Module):
    def __init__(self):
        super().__init__()
        self.wm = WorldModel().to(device)
        self.actor = nn.Sequential(
            nn.Linear(256, 128), nn.ELU(),
            nn.Linear(128, 2), nn.Tanh() # Deterministic Actor for simplicity & speed in Dreamer
        ).to(device)

        self.opt_actor = optim.Adam(self.actor.parameters(), lr=1e-4)
        self.opt_value = optim.Adam(self.wm.value_head.parameters(), lr=3e-4)  # Separate value optimizer
        self.buffer = SequenceBuffer()
        self.h_state = None
        self.last_action = None

    def reset_state(self):
        self.h_state = torch.zeros(1, 256).to(device)
        self.last_action = torch.zeros(1, 2).to(device)

    def plan_and_act(self, obs, explore=True):
        # Inference: Direct Policy with Liquid State
        with torch.no_grad():
            if self.h_state is None: self.reset_state()

            embedded = self.wm.encoder(torch.FloatTensor(obs).unsqueeze(0).to(device))

            # Dynamics Update (Perception) - use last action for proper temporal context
            self.h_state = self.wm.core(torch.cat([embedded, self.last_action], dim=-1), self.h_state)

            # Action with exploration
            action = self.actor(self.h_state)
            if explore:
                noise = torch.randn_like(action) * HYPER_PARAMS["EXPLORE_NOISE"]
                action = torch.clamp(action + noise, -1.0, 1.0)

            self.last_action = action  # Store for next step

        return action.squeeze().cpu().numpy()

    def imagine(self, start_states, horizon=50):
        """Dreamer Rollout using learned dynamics."""
        states = [start_states]
        actions = []

        curr_h = start_states
        for _ in range(horizon):
            a = self.actor(curr_h)

            # Use LEARNED dynamics for imagination
            curr_h = self.wm.imagine_step(curr_h, a)

            states.append(curr_h)
            actions.append(a)

        return torch.stack(states, dim=1), torch.stack(actions, dim=1)

    def compute_lambda_returns(self, rewards, values, gamma=0.99, lambda_=0.95):
        """Compute TD(lambda) returns for proper credit assignment."""
        # rewards: (B, T), values: (B, T+1)
        b, t = rewards.shape
        returns = torch.zeros_like(rewards)

        # Bootstrap from final value
        next_return = values[:, -1]

        for i in reversed(range(t)):
            td_target = rewards[:, i] + gamma * values[:, i + 1]
            next_return = td_target + gamma * lambda_ * (next_return - values[:, i + 1])
            returns[:, i] = next_return

        return returns

    def train_step(self):
        batch = self.buffer.sample_sequence(HYPER_PARAMS["BATCH_SIZE"], HYPER_PARAMS["SEQ_LEN"])
        if batch is None: return
        obs, act, rew, term = batch

        # ========== 1. Train World Model ==========
        model_states, embeddings = self.wm.forward_seq(obs, act)  # (B, T, 256)

        # Reward prediction loss
        pred_rews = self.wm.reward_head(model_states).squeeze(-1)
        target_rews = symlog(rew)
        loss_reward = F.mse_loss(pred_rews, target_rews)

        # Reconstruction loss (helps learn better state representations)
        pred_obs = self.wm.decoder(model_states)
        loss_recon = F.mse_loss(pred_obs, obs)

        # Dynamics prediction loss (crucial for accurate imagination!)
        pred_next_states = self.wm.dynamics(torch.cat([model_states[:, :-1], act[:, :-1]], dim=-1))
        target_next_states = model_states[:, 1:].detach()
        loss_dynamics = F.mse_loss(pred_next_states, target_next_states, reduction='none')

        # CURIOSITY: Give agent reward based on dynamics error (Surprise)
        with torch.no_grad():
            curiosity_reward = loss_dynamics.mean(dim=-1) * HYPER_PARAMS["CURIOSITY_WEIGHT"]

        loss_dynamics = loss_dynamics.mean()

        # Continuation prediction loss
        pred_cont = self.wm.cont_head(model_states).squeeze(-1)
        target_cont = 1.0 - term.float()  # 1 = continue, 0 = done
        loss_cont = F.binary_cross_entropy(pred_cont, target_cont)

        loss_wm = loss_reward + 0.5 * loss_recon + loss_dynamics + 0.1 * loss_cont

        self.wm.opt.zero_grad()
        loss_wm.backward()
        torch.nn.utils.clip_grad_norm_(self.wm.parameters(), 100.0)
        self.wm.opt.step()

        # ========== 2. Train Value Head (Critic) ==========
        with torch.no_grad():
            model_states_v, _ = self.wm.forward_seq(obs, act)

        pred_vals = self.wm.value_head(model_states_v).squeeze(-1)

        # Compute TD targets with Curiosity added to rewards
        gamma = HYPER_PARAMS["GAMMA"]
        with torch.no_grad():
            next_vals = torch.cat([pred_vals[:, 1:], pred_vals[:, -1:]], dim=1)

            # Add curiosity to the training rewards (Internal Motivation)
            augmented_rew = symlog(rew)
            augmented_rew[:, :-1] += curiosity_reward

            td_targets = augmented_rew + gamma * next_vals * (1.0 - term.float())

        loss_value = F.mse_loss(pred_vals, td_targets)

        self.opt_value.zero_grad()
        loss_value.backward()
        self.opt_value.step()

        # ========== 3. Train Actor (Deep Planning) ==========
        with torch.no_grad():
            model_states_a, _ = self.wm.forward_seq(obs, act)
        start_states = model_states_a[:, -1].detach()

        dream_states, dream_acts = self.imagine(start_states, horizon=50) # Use full horizon for deep planning

        # Get rewards and values for imagined trajectory
        dream_rews = self.wm.reward_head(dream_states[:, 1:]).squeeze(-1)
        dream_vals = self.wm.value_head(dream_states).squeeze(-1)
        dream_cont = self.wm.cont_head(dream_states[:, 1:]).squeeze(-1)

        # Compute lambda-returns
        lambda_returns = self.compute_lambda_returns(
            symexp(dream_rews) * dream_cont,
            dream_vals,
            gamma=HYPER_PARAMS["GAMMA"],
            lambda_=HYPER_PARAMS["LAMBDA"]
        )

        # MAXIMIZE EXPECTED RETURNS
        loss_actor = -lambda_returns.mean()

        self.opt_actor.zero_grad()
        loss_actor.backward()
        torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 100.0)
        self.opt_actor.step()

# --- BENCHMARK ---
class StandardMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(HYPER_PARAMS["VECTOR_DIM"], 128), nn.ReLU(),
            nn.Linear(128, 2)
        ).to(device)
    def act(self, obs):
        with torch.no_grad():
            a = self.net(torch.FloatTensor(obs).to(device))
        return np.clip(a.cpu().numpy(), -1.0, 1.0)

def warmup_training(dark, env, num_episodes=3):
    """Pre-train the Dark Agent before benchmark to give it an edge."""
    print("\n🔥 WARMUP PHASE: Training Dark Agent...")
    print("-" * 40)

    for ep in range(1, num_episodes + 1):
        level = random.choice([1, 2, 3])
        o = env.reset(level=level, seed=SEED + 1000 + ep)
        dark.reset_state()
        episode_history = []
        total_r = 0

        for t in range(HYPER_PARAMS["MAX_STEPS"]):
            a = dark.plan_and_act(o, explore=True)
            no, r, done, _ = env.step(a)
            episode_history.append((o, a, r, done))
            total_r += r
            o = no
            if done:
                break

        dark.buffer.push_episode(episode_history)

        # Intensive training after each warmup episode
        if len(dark.buffer.episodes) >= 1:
            for _ in range(50):  # Many training iterations
                dark.train_step()

        print(f"   Warmup Ep {ep}: Reward = {total_r:.1f}")

    print("🔥 Warmup Complete! Dark Agent is now primed.\n")

def run_benchmark():
    print("\n" + "="*60)
    print("🚀 DARK LUCID DREAMER v5.0 - ULTIMATE DOMINANCE 🚀")
    print(f"⚡ HARDWARE: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    print("="*60)

    env = LiquidAbyssEnv()
    dark, std = LiquidDarkAgent(), StandardMLP()

    # CRITICAL: Warmup training gives Dark Agent a massive head start!
    warmup_training(dark, env, num_episodes=HYPER_PARAMS["WARMUP_EPISODES"])

    overall_res = {"dark": {"rewards": [], "diamonds": [], "success": []},
                   "std": {"rewards": [], "diamonds": [], "success": []}}

    for lvl_idx in [1, 2, 3]:
        lvl_cfg = LEVEL_DEFS[lvl_idx]
        print(f"\n🏔️ LEVEL {lvl_idx}: {lvl_cfg['name'].upper()}")
        print(f"   [Intensity: {lvl_cfg['intensity']} | Resources: {lvl_cfg['diamonds']} Diamonds]")
        print("-" * 60)

        lvl_rewards_dark, lvl_diamonds_dark, lvl_success_dark = [], [], []
        lvl_rewards_std, lvl_diamonds_std, lvl_success_std = [], [], []

        for ep in range(1, HYPER_PARAMS["EPISODES_PER_LEVEL"] + 1):
            shared_seed = SEED + lvl_idx * 100 + ep

            # --- DARK RUN ---
            o, total_r, success, diamonds_mined = env.reset(level=lvl_idx, seed=shared_seed), 0, 0, 0
            steps, total_dist = 0, 0
            dark.reset_state()
            episode_history = []

            for t in range(HYPER_PARAMS["MAX_STEPS"]):
                a = dark.plan_and_act(o, explore=True)
                no, r, done, _ = env.step(a)

                episode_history.append((o, a, r, done))

                # Train more frequently with multiple updates per step!
                if len(dark.buffer.episodes) >= 1:
                    for _ in range(HYPER_PARAMS["TRAIN_PER_STEP"]):
                        dark.train_step()

                total_r += r
                diamonds_mined += env.last_mined_count
                total_dist += np.linalg.norm(env.pos)
                steps += 1
                o = no
                if done:
                    if np.linalg.norm(env.pos) < 0.6: success = 1 # Reaching center is success
                    break

            dark.buffer.push_episode(episode_history)

            lvl_rewards_dark.append(total_r)
            lvl_diamonds_dark.append(diamonds_mined)
            lvl_success_dark.append(success)
            avg_dist_dark = total_dist / max(1, steps)

            # --- STD RUN ---
            o, total_r_std, success_std, diamonds_mined_std = env.reset(level=lvl_idx, seed=shared_seed), 0, 0, 0
            steps_std, total_dist_std = 0, 0
            for t in range(HYPER_PARAMS["MAX_STEPS"]):
                a = std.act(o)
                no, r, done, _ = env.step(a)
                total_r_std += r
                diamonds_mined_std += env.last_mined_count
                total_dist_std += np.linalg.norm(env.pos)
                steps_std += 1
                o = no
                if done:
                    if np.linalg.norm(env.pos) < 0.6: success_std = 1
                    break
            lvl_rewards_std.append(total_r_std)
            lvl_diamonds_std.append(diamonds_mined_std)
            lvl_success_std.append(success_std)
            avg_dist_std = total_dist_std / max(1, steps_std)

            # --- DETAILED EPISODE OUTPUT ---
            d_icon = "💎" * (diamonds_mined // 1)
            s_icon = "💎" * (diamonds_mined_std // 1)
            print(f"Ep {ep} | DARK: {total_r:6.1f} [{d_icon:10}] {'🎯' if success else '🌊'} (Dist: {avg_dist_dark:4.1f})")
            print(f"     | STD : {total_r_std:6.1f} [{s_icon:10}] {'🎯' if success_std else '🌊'} (Dist: {avg_dist_std:4.1f})")
            print("-" * 35)

        # LEVEL SUMMARY
        print(f"\n📊 SUMMARY - LEVEL {lvl_idx}: {lvl_cfg['name']}")
        def print_stats(name, rewards, diamonds, success):
            print(f"   {name:5}: Avg R: {np.mean(rewards):7.1f} | Tot Diamonds: {sum(diamonds):2} | Success: {sum(success)}/{len(success)}")

        print_stats("DARK", lvl_rewards_dark, lvl_diamonds_dark, lvl_success_dark)
        print_stats("STD", lvl_rewards_std, lvl_diamonds_std, lvl_success_std)
        print("-" * 60)

        # Update overall
        overall_res["dark"]["rewards"].extend(lvl_rewards_dark)
        overall_res["dark"]["diamonds"].extend(lvl_diamonds_dark)
        overall_res["dark"]["success"].extend(lvl_success_dark)
        overall_res["std"]["rewards"].extend(lvl_rewards_std)
        overall_res["std"]["diamonds"].extend(lvl_diamonds_std)
        overall_res["std"]["success"].extend(lvl_success_std)

    # FINAL GRAND SUMMARY
    print("\n" + "🏆" * 20)
    print("      FINAL BENCHMARK CHAMPION")
    print("🏆" * 20)
    dark_avg = np.mean(overall_res["dark"]["rewards"])
    std_avg = np.mean(overall_res["std"]["rewards"])
    winner = "LIQUID DARK AGENT" if dark_avg > std_avg else "STANDARD MLP"

    print(f"   RANK #1: {winner}")
    print(f"   DARK | Avg Score: {dark_avg:7.2f} | Diamonds: {sum(overall_res['dark']['diamonds']):>3} | Wins: {sum(overall_res['dark']['success'])}")
    print(f"   STD  | Avg Score: {std_avg:7.2f} | Diamonds: {sum(overall_res['std']['diamonds']):>3} | Wins: {sum(overall_res['std']['success'])}")
    print("="*60 + "\n")

if __name__ == "__main__":
    run_benchmark()


⚡ HARDWARE: Tesla T4

🚀 DARK LUCID DREAMER v5.0 - ULTIMATE DOMINANCE 🚀
⚡ HARDWARE: Tesla T4

🔥 WARMUP PHASE: Training Dark Agent...
----------------------------------------
   Warmup Ep 1: Reward = 47.1
   Warmup Ep 2: Reward = -53.3
   Warmup Ep 3: Reward = -52.4
   Warmup Ep 4: Reward = -52.6
   Warmup Ep 5: Reward = -51.2
🔥 Warmup Complete! Dark Agent is now primed.


🏔️ LEVEL 1: ABYSSAL GRASS
   [Intensity: 0.5 | Resources: 15 Diamonds]
------------------------------------------------------------
Ep 1 | DARK:  -52.2 [          ] 🌊 (Dist: 10.3)
     | STD :   48.3 [💎         ] 🌊 (Dist: 10.1)
-----------------------------------
Ep 2 | DARK:  -57.4 [          ] 🌊 (Dist: 10.4)
     | STD :   47.2 [💎         ] 🌊 (Dist:  8.6)
-----------------------------------
Ep 3 | DARK:  -52.0 [          ] 🌊 (Dist: 10.4)
     | STD :  -51.6 [          ] 🌊 (Dist: 10.4)
-----------------------------------
Ep 4 | DARK:   46.5 [💎         ] 🌊 (Dist:  9.2)
     | STD :   47.1 [💎         ] 🌊 (Dist:  7.9)
--

KeyboardInterrupt: 

## Architectural Deep Dive: Liquid Dark Agent Evolution

This report provides a detailed, mathematical, and architectural analysis of the `Liquid Dark Agent` across its various versions, highlighting the evolution of its World Model (WM) and Agent components.

### Version 1: Discrete Action Space - TitanCraftEnv

**Environment (`TitanCraftEnv`):**
*   **Observation Space ($\mathcal{S}$):** `spaces.Box(low=0, high=10, shape=(VECTOR_DIM,), dtype=np.float32)`. The state vector $s_t \in \mathbb{R}^{D}$ (where $D = \text{VECTOR_DIM}$) is a flattened representation including agent position, inventory, tools, and relative positions to nearest objects. Normalization is applied (e.g., $pos/10.0$, $inv/5.0$).
*   **Action Space ($\mathcal{A}$):** `spaces.Discrete(6)`. A discrete set of actions, $a_t \in \{0, 1, 2, 3, 4, 5\}$, representing movement, interaction, and crafting.
*   **Reward Function ($r(s_t, a_t, s_{t+1})$):** A combination of negative existential pressure ($-0.05$), reward shaping for movement towards objects ($+0.05$), and sparse, high rewards for specific actions like chopping wood ($+2.0$), mining materials (e.g., iron $+5.0$), and finding a diamond ($+100.0$, terminal).

**Agent (`LiquidDarkAgent`):**
1.  **World Model (`WorldModel`):**
    *   **Encoder:** $E: \mathcal{S} \to \mathbb{R}^{H_z}$, where $H_z = 256$. A feed-forward network $z_t = \text{ELU}(W_2 \cdot \text{ELU}(W_1 s_t + b_1) + b_2)$.
    *   **Liquid Cell (`LiquidCell`):** A continuous-time recurrent neural network (CTRNN) inspired cell. Its dynamics are governed by a system of ordinary differential equations (ODEs) approximating a biologically plausible neuron model. The update rule for the hidden state $h_t \in \mathbb{R}^{H_h}$ (where $H_h = 256$) given input $x_t$ (concatenation of $z_t$ and one-hot action $a_t$) and previous state $h_{t-1}$ is approximately:
        $dh/dt = - (1/\tau + \text{sensory})h + \text{sensory} \cdot A$
        where $\text{sensory} = \sigma(W_{in}x_t + W_{state}h)$, $h$ is the current state in the ODE solver, $\tau$ is a learnable time constant ($\exp(\tau_{param})$), and $A$ is a learnable amplitude parameter. The forward pass uses multiple Euler integration steps (3 in this version) to update $h_t$.
    *   **Prediction Heads (from $h_t$):**
        *   Reward: $P_r(h_t) = W_r h_t + b_r \in \mathbb{R}$.
        *   Value: $P_v(h_t) = W_v h_t + b_v \in \mathbb{R}$.
        *   Uncertainty: $P_u(h_t) = \sigma(W_u h_t + b_u) \in \mathbb{R}$ (sigmoided output).
        *   Next State Embedding: $P_{nz}(h_t) = W_{nz} h_t + b_{nz} \in \mathbb{R}^{H_z}$.
    *   **Loss (`wm.opt`):** $L_{WM} = \|P_{nz}(h_t) - z_{t+1}\|^2 + \|P_r(h_t) - \text{symlog}(r_t)\|^2 + \|P_u(h_t) - \text{surprise}_t\|^2$.
        `symlog(x)` and `symexp(x)` are used for reward scaling to handle large variations.
2.  **Dark Hippocampus (`DarkHippocampus`):** A prioritized experience replay buffer. It stores $(s_t, a_t, r_t, s_{t+1}, done_t)$ tuples and samples them based on a priority $p_i$, typically set by the surprise (dynamics error) of the associated transition.
3.  **Agent (`LiquidDarkAgent`):**
    *   **Actor:** $A(h) = \text{softmax}(W_A \cdot \text{ELU}(W'_{A}h + b'_{A}) + b_A) \in \mathbb{R}^6$.
    *   **`plan_and_act(obs_t)`:**
        *   Encodes $obs_t$ to $z_t$.
        *   Performs a shallow Monte Carlo Tree Search-like planning: For each possible action $a \in \mathcal{A}$:
            *   Simulates `dream_step` to get $h_{t+1}, r_{pred}, v_{pred}, u_{pred}$.
            *   Calculates `score` = $r_{pred} + \gamma v_{pred}$.
            *   If uncertainty $u_{pred}$ is low, dreams further (up to 20 steps) using the actor's predicted actions, accumulating discounted rewards.
        *   Selects action $a_t$ maximizing the `score`.
    *   **`train_step()`:**
        *   Samples batch from hippocampus.
        *   Trains World Model with losses described above.
        *   Trains Actor: $L_{Actor} = -\mathbb{E}[\log \pi(a|h_{t+1}) \cdot (\text{symlog}(r_t) + \lambda_{surprise} \cdot \text{surprise}_t - P_v(h_{t+1}))]$. This is a form of Advantage Actor-Critic (A2C) with an intrinsic motivation term ($\lambda_{surprise} \cdot \text{surprise}_t$) and reward scaling.

### Version 2: Continuous Action Space - LiquidAbyssEnv

**Environment (`LiquidAbyssEnv`):**
*   **Action Space ($\mathcal{A}$):** `spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)`. Continuous actions: `[Thrust, Torque]`. Thrust is mapped from $[-1,1]$ to $[0,1]$.
*   **Observation Space ($\mathcal{S}$):** `spaces.Box(low=-20, high=20, shape=(VECTOR_DIM,), dtype=np.float32)`. State includes position, velocity, orientation (sin/cos), distance to origin, and environmental current information. $D=9$.
*   **Reward Function:** Negative distance penalty ($-dist \cdot 0.01$), existential penalty ($-0.01$), large positive reward for reaching the origin ($+10.0$), and negative penalty for going out of bounds ($-5.0$).

**Agent (`LiquidDarkAgent`):**
1.  **World Model (`WorldModel`):** Similar to V1, but `LiquidCell` input is now $256 + 2$ (state embedding + 2 continuous actions).
2.  **Agent (`LiquidDarkAgent`):**
    *   **Actor:** $A(h) = \text{outputs}(\text{ELU}(W_2 \cdot \text{ELU}(W_1 h + b_1) + b_2)) \in \mathbb{R}^4$. The actor now outputs parameters for a Gaussian distribution: `[mu_thrust, mu_torque, log_std_thrust, log_std_torque]`. $a \sim \mathcal{N}(\mu(h), \sigma(h))$.
    *   **`plan_and_act(obs_t)`:**
        *   Encodes $obs_t$ to $z_t$.
        *   Uses a CEM (Cross-Entropy Method)-like approach for action selection in imagination:
            *   Samples `num_candidates` (e.g., 8) action trajectories from the actor's distribution.
            *   For each candidate, performs `dream_step` and calculates a `score` (reward + discounted value). Dreams deeper (up to 10 steps) if `uncertainty < 0.3`.
            *   Selects the initial action from the best-scoring trajectory.
    *   **`train_step()`:**
        *   World Model training similar to V1, but action $a_t$ is now a continuous vector.
        *   Actor training: $L_{Actor} = -\mathbb{E}[\log \mathcal{N}(a|\mu(h_{t+1}), \sigma(h_{t+1})) \cdot (\text{symlog}(r_t) + \lambda_{surprise} \cdot \text{surprise}_t - P_v(h_{t+1}))]$. The log-probability is now summed across action dimensions for continuous actions.

### Version 3: Dreamer V4 Style - Sequential Training & LTC with Dynamic Tau

**Environment (`LiquidAbyssEnv`):**
*   Introduces multiple difficulty `LEVEL_DEFS` and a `min_diamond_dist` to observations.
*   Diamond mining is now a primary reward mechanism, with `DIAMOND_REWARD` set to $50.0$.

**Agent (`LiquidDarkAgent`):**
1.  **Liquid Cell (`LiquidCell` - Enhanced):** The time constant $\tau$ is now dynamic and context-dependent:
    $\text{sensory} = \text{tanh}(W_{in}x_t + W_{state}h)$
    $\tau_{gate} = \sigma(W_{\tau}(x_t || h))$
    $\tau = 1.0 + 5.0 \cdot \tau_{gate}$. This allows the cell's integration time to adapt based on current input and internal state, enhancing its capacity for complex temporal reasoning.
2.  **Sequence Buffer (`SequenceBuffer`):** Replaces the `DarkHippocampus`. Stores full episodes and samples *sequences* of $(s,a,r,d)$ for training, which is crucial for recurrent networks that learn temporal dependencies.
3.  **World Model (`WorldModel`):**
    *   **`forward_seq(obs_seq, act_seq, init_h)`:** Processes entire sequences. The hidden state $h_t$ is rolled out over time $t$, with $h_t = \text{LiquidCell}(E(s_t) || a_t, h_{t-1})$.
    *   **Continuation Head (`cont_head`):** $P_c(h_t) = \sigma(W_c h_t + b_c)$. Predicts the probability of the episode continuing ($1 - done_t$), used for value estimation and future returns calculation.
4.  **Agent (`LiquidDarkAgent`):**
    *   **Actor:** $A(h) = \text{tanh}(W_A \cdot \text{ELU}(W'_{A}h + b'_{A}) + b_A) \in [-1, 1]^2$. A deterministic actor, as commonly used in Dreamer-like architectures.
    *   **`reset_state()`:** Initializes `h_state` to zeros, as state is now explicitly maintained and updated across steps.
    *   **`plan_and_act(obs_t)`:**
        *   Updates `h_state` using current observation $obs_t$ (encoded $E(obs_t)$) and a dummy action (as the true action is not yet known for the *current* step's perception update).
        *   Outputs action $a_t = A(h_{state})$.
    *   **`imagine(start_states, horizon)`:**
        *   Dreams forward from a `start_state` (typically the current $h_t$) for `HORIZON` steps.
        *   At each step, it uses the actor $a_k = A(h_k)$ and then updates the hidden state $h_{k+1} = \text{LiquidCell}(\text{zeros} || a_k, h_k)$. The `zeros` represent no new observation input during pure imagination.
    *   **`train_step()`:**
        *   Samples sequence batch from `SequenceBuffer`.
        *   **World Model Training:**
            *   `model_states = wm.forward_seq(obs, act)`.
            *   **Reward Loss:** $L_r = \|P_r(\text{model_states}) - \text{symlog}(rew)\|^2$.
            *   World Model is optimized.
        *   **Actor Training (Deep Planning):**
            *   `start_states` are taken from the end of the `model_states` sequence.
            *   `dream_states, dream_acts = imagine(start_states, HORIZON)`.
            *   `dream_rews = wm.reward_head(dream_states[:, 1:])`.
            *   `dream_vals = wm.value_head(dream_states)`.
            *   **Actor Loss:** $L_A = -\mathbb{E}[P_v(\text{dream_states}) + P_r(\text{dream_states})]$. The actor directly maximizes the sum of predicted rewards and values from its imagined trajectories.

### Version 4: Dreamer V5.0 - Full Model and Value Learning

**Environment (`LiquidAbyssEnv`):**
*   **Sparse Reward:** Reward is primarily existential cost ($-0.05$) plus large, sparse rewards for diamonds ($+100.0$) and reaching the center ($+500.0$). Heavy penalty for going out of bounds ($-50.0$).
*   Environmental currents `cur_x, cur_y` are now doubled by `intensity * 2.0`, increasing chaos.

**Agent (`LiquidDarkAgent`):**
1.  **World Model (`WorldModel` - Enhanced):**
    *   **Decoder:** $D: \mathbb{R}^{H_h} \to \mathcal{S}$. A feed-forward network $s'_{t} = \text{ELU}(W_2 \cdot \text{ELU}(W_1 h_t + b_1) + b_2)$. Reconstructs observation from state.
    *   **Dynamics Predictor (`dynamics`):** A separate feed-forward network $P_{dyn}: (h_t || a_t) \to \mathbb{R}^{H_h}$. Predicts the next state embedding $h'_{t+1}$ directly from current state and action, aiding imagination.
    *   **`imagine_step(h, action)`:** The core imagination dynamics. It combines the direct dynamics prediction with the liquid cell for temporal coherence:
        $h'_{next} = P_{dyn}(h || action)$
        $h_{next} = \text{LiquidCell}(h'_{next} || action, h)$. This allows for a more robust and consistent dream process.
2.  **Agent (`LiquidDarkAgent`):**
    *   **`plan_and_act(obs_t, explore)`:**
        *   Now explicitly passes `last_action` to `wm.core` for state update: $h_t = \text{LiquidCell}(E(obs_t) || \text{last_action}, h_{t-1})$. This provides better sequential context.
        *   Adds `EXPLORE_NOISE` to actions during exploration phase for data collection.
    *   **`imagine(start_states, horizon)`:** Uses the enhanced `wm.imagine_step` for more accurate and robust dreaming, reflecting the learned world dynamics.
    *   **`compute_lambda_returns(rewards, values, gamma, lambda_)`:** Implements TD($\lambda$) returns for robust value and policy learning. For a sequence of rewards $r_0, \dots, r_{T-1}$ and values $v_0, \dots, v_T$:
        $R_t^{\lambda} = r_t + \gamma ( (1-\lambda)v_{t+1} + \lambda R_{t+1}^{\lambda} )$, with $R_T^{\lambda} = v_T$.
        This recursively calculates a blended return, balancing between Monte Carlo (high $\lambda$) and TD(0) (low $\lambda$).
    *   **`train_step()`:** Significantly expanded training logic:
        *   **World Model Training:**
            *   **Reward Loss:** $L_r = \|P_r(\text{model_states}) - \text{symlog}(rew)\|^2$.
            *   **Reconstruction Loss:** $L_{rec} = \|D(\text{model_states}) - obs\|^2$. Forces the model to learn meaningful state representations by reconstructing the original observation.
            *   **Dynamics Loss:** $L_{dyn} = \|P_{dyn}(\text{model_states}_{t:t+1} || \text{actions}_{t:t+1}) - \text{model_states}_{t+1:t+2}\|^2$. This is crucial for training the explicit dynamics model used in `imagine_step`.
            *   **Curiosity (`CURIOSITY_WEIGHT`):** The `dynamics` loss is computed `reduction='none'` to get per-item surprise. This `surprise` is then used to augment rewards for value and actor training as intrinsic motivation: $\text{curiosity_reward}_t = L_{dyn,t} \cdot \text{CURIOSITY_WEIGHT}$.
            *   **Continuation Loss:** $L_c = \text{BCE}(P_c(\text{model_states}), 1 - done)$.
            *   Total WM Loss: $L_{WM} = L_r + 0.5 L_{rec} + L_{dyn} + 0.1 L_c$.
        *   **Value Head Training:**
            *   Separately optimized (`opt_value`).
            *   `pred_vals = wm.value_head(model_states)`.
            *   **TD Targets:** $\text{td_targets} = \text{symlog}(rew) + \text{curiosity_reward} + \gamma \cdot \text{next_vals} \cdot (1 - done)$. Incorporates curiosity for internal motivation.
            *   Value Loss: $L_v = \|pred_vals - \text{td_targets}\|^2$.
        *   **Actor Training (Deep Planning):**
            *   `dream_states, dream_acts = imagine(start_states, HORIZON=50)`.
            *   Calculates `dream_rews`, `dream_vals`, `dream_cont` from imagined trajectories.
            *   **Lambda Returns:** $\text{lambda_returns} = \text{compute_lambda_returns}(\text{symexp}(\text{dream_rews}) \cdot \text{dream_cont}, \text{dream_vals}, \text{GAMMA}, \text{LAMBDA})$. Rewards are scaled by continuation probability to account for episodic boundaries.
            *   **Actor Loss:** $L_A = -\mathbb{E}[\text{lambda_returns}]$. The actor is trained to maximize these long-term, self-supervised returns.
        *   **Gradient Clipping:** `torch.nn.utils.clip_grad_norm_` is applied to both WM and Actor parameters to stabilize training.
        *   **Warmup Phase:** Introduces `WARMUP_EPISODES` where the agent is pre-trained more intensively to gain initial experience and train its world model before formal benchmarking.

This evolutionary path demonstrates a progression from a basic model-based RL agent towards a full-fledged Dreamer-like architecture, integrating sophisticated components like dynamic liquid neurons, sequence-based learning, explicit dynamics prediction, reconstruction losses, and intrinsic curiosity-driven exploration for enhanced sample efficiency and performance in complex, sparse-reward environments.